# 卷积神经网络分类图片过程详解

在这一节中，你将学习如何对 CIFAR-10 中的图片进行分类。

CIFAR-10 数据集由 10 类 60000 张 32×32像素的彩色图片组成，每类有 6000 张图片。有 50000 张训练图片和 10000 张测试图片。下面的图片取自https://www.cs.toronto.edu/~kriz/cifar.html：
![](./Markdown_images/miao_28_TensorFlow_卷积神经网络分类图片过程详解/01.jpg)
在这一节，将使用 TFLearn（一个更高层次的框架），它抽象了一些 TensorFlow 的内部细节，能够专注于深度网络的定义。可以在 http://tflearn.org/ 上了解 TFLearn 的信息，这里的代码是标准发布的一部分，网址为https://github.com/tflearn/tflearn/tree/master/examples。

# 具体操作过程

1. 导入几个 utils 和核心层用于实现 ConvNet、dropout、fully_connected 和 max_pool。另外，导入一些对图像处理和图像增强有用的模块。请注意，TFLearn 为 ConvNet 提供了一些已定义的更高级别的层，这能够专注于代码的定义：

In [1]:
import tflearn
from tflearn.datasets import cifar10  # 加载CIFAR-10数据集
from tflearn.data_utils import shuffle, to_categorical  # 混洗 独热
from tflearn.data_augmentation import ImageAugmentation  # 数据增强
from tflearn.data_preprocessing import ImagePreprocessing  # 数据预处理
from tflearn.layers.conv import conv_2d, max_pool_2d  # CNN 池化
from tflearn.layers.core import input_data, dropout, fully_connected  # 输入数据 dropout 全连接
from tflearn.layers.estimator import regression  # 回归

D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


2. 加载 CIFAR-10 数据，并将其分为 X_train 和 Y_train，X_test 用于测试，Y_test 是测试集的标签。对 X 和 Y 进行混洗可能是有用的，因为这样能避免训练依赖于特定的数据配置。最后一步是对 X 和 Y 进行独热编码：

In [2]:
# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data('../Database/')
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

3. 使用 ImagePreprocessing() 对数据集进行零中心化（即对整个数据集计算平均值），同时进行 STD 标准化（即对整个数据集计算标准差）。TFLearn 数据流旨在通过 CPU 先对数据进行预处理，然后在 GPU 上加速模型训练：

In [3]:
# Real -time data Preprocessing
img_prep = ImagePreprocessing()  #预处理
img_prep.add_featurewise_zero_center()  # 零中心化
img_prep.add_featurewise_stdnorm()  # 对整个数据集计算标准差 进行 STD 标准化

4. 通过随机左右翻转和随机旋转来增强数据集。这一步是一个简单的技巧，用于增加可用于训练的数据：

In [4]:
# Real - Time data augmention
img_aug = ImageAugmentation()  # 数据增强
img_aug.add_random_flip_leftright()  #数据增强 随机左右翻转
img_aug.add_random_rotation(max_angle=25.0)  # 随机旋转 25 度

5. 使用之前定义的图片预处理和图片增强操作创建卷积网络。网络由三个卷积层组成。第一层有 32 个卷积核，尺寸是 3×3，激活函数用 ReLU，这一层后使用 max_pool 层用于缩小尺寸。然后是两个卷积核级联，卷积核的个数是 64，尺寸是 3×3，激活函数是 ReLU。之后依次是 max_pool 层，具有 512 个神经元、激活函数为 ReLU 的全连接的网络，设置 dropout 概率为 50%。最后一层是全连接层，利用 10 个神经元和激活函数 softmax 对 10 个手写数字进行分类。请注意，这种特殊类型的 ConvNet 在 CIFAR-10 中非常有效。其中，使用 Adam 优化器（categorical_crossentropy）学习率是 0.001：

In [5]:
#Convolutional network building
network = input_data(
    shape=[None, 32, 32, 3],  # 输入的图像为 32X32 3 通道
    data_preprocessing=img_prep,
    data_augmentation=img_aug)

network = conv_2d(network, 32, 3,activation='relu')# 3X3 的卷积核 输出 32通道 使用 relu 激活函数
network = max_pool_2d(network, 2)# 池化窗口 2X2 使用 relu 激活函数 图像尺寸变为原来的一半 16X16

network = conv_2d(network, 64, 3, activation='relu')# 3X3 的卷积核 输出 64 通道 使用 relu 激活函数
network = conv_2d(network, 64, 3, activation='relu')# 3X3 的卷积核 输出 64 通道 使用 relu 激活函数
network = max_pool_2d(network, 2)# 池化窗口 2X2 使用 relu 激活函数 图像尺寸变为原来的一半 8X8

network = fully_connected(network, 512, activation='relu') # 全连接层 有 512个节点
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')#最后10个输出，然后使用 softmax 处理
network = regression(
    network,
    optimizer='adam',
    loss='categorical_crossentropy',  # 分类的交叉熵损失函数
    learning_rate=0.001)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


6. 实例化 ConvNet 并以 batch_size=96 训练 50 个 epoch：

In [6]:
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X,
          Y,
          n_epoch=50,
          shuffle=True,
          validation_set=(X_test, Y_test),
          show_metric=True, # 显示指标
          batch_size=96,
          run_id='cifar10_cnn')

Training Step: 26049  | total loss: 0.40044 | time: 24.205s
| Adam | epoch: 050 | loss: 0.40044 - acc: 0.8628 -- iter: 49920/50000
Training Step: 26050  | total loss: 0.38497 | time: 25.876s
| Adam | epoch: 050 | loss: 0.38497 - acc: 0.8682 | val_loss: 0.68778 - val_acc: 0.8125 -- iter: 50000/50000
--
